In [3]:
from __future__ import absolute_import, division, print_function
import os, sys, json
current_path = os.path.abspath('.')
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torchvision import datasets, transforms
from tqdm import tqdm
from torchinfo import summary

from src.data_loader import *
from src.utils import *

from src.models.simple_cnn import *
from src.models.resnet_models import *
from src.models.dscnn import *
from train_joint import *

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

############################################
########## Plot Style Declaration ##########
# Set the style globally
# Alternatives include bmh, fivethirtyeight, ggplot,
# dark_background, seaborn-deep, etc
# plt.style.use('ggplot')
plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'times new roman'
# plt.rcParams['font.serif'] = 'Ubuntu'
# plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

##################################
########## End of Setup ##########

import tensorflow as tf

In [4]:
class ArgClass():
    def __init__(self):
        super(ArgClass, self).__init__()

args = ArgClass()
args.dataset="mnist"
args.model_arch = "cnn"
args.batch_size=128
args.test_batch_size=1000
args.percent=[0.8, 0.92, 0.991, 0.93]
args.alpha=5e-4
args.rho=1e-2
args.l1=False
args.l2=False
args.num_pre_epochs=3
args.num_epochs=10
args.num_re_epochs=3
args.lr=1e-3
args.adam_epsilon=1e-8
args.no_cuda=False
args.seed=1
args.save_model=False
args.shuffle=True

args.optimizer_name = 'adam'
args.lr_mode = 'multistep'
args.lr_decay = 0.1
args.lr_decay_epoch = '20,40'
args.target_lr = 1e-8
args.warmup_epochs = 0
args.warmup_lr = 1e-8
args.warmup_mode = 'linear'
args.momentum = 0.9
args.wd = 0.0001
args.gamma_wd_mult = 1.0
args.beta_wd_mult = 1.0
args.bias_wd_mult = 1.0
args.grad_clip = None
args.label_smoothing = False


args.test_fold_l = '[10]'
args.use_one_task = 'false'
args.exp_setup = ''
args.subject_idx = None
args.pretrained=False
args.best_acc = 200.0
args.mixup = False
args.mixup_alpha = 1.0
args.mixup_epoch_tail = 10
args.session = 1
args.test_vote = None

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [12]:
# MicroNet-AD-K
init_block_channel = 176
init_block_stride = (2,2)
channels = [[192],[192],[192],[192],[192]]
strides = [2, 1, 1, 2, 2]

def get_pool_size(strides, config):
    pool_size = None
    for stride in strides:
        if stride > 1:
            if pool_size is None:
                pool_size=[np.ceil(config["seq"]/2),np.ceil(config["features"]/2)]
            else:
                pool_size = [np.ceil(pool_size[0]/2),np.ceil(pool_size[1]/2)]
    # return pool_size
    return [np.ceil(pool_size[0]/2),np.ceil(pool_size[1]/2)]


In [7]:
print(tf.keras.backend.image_data_format())
# force channels-first ordering
# tf.keras.backend.set_image_data_format('channels_first')
# print(tf.keras.backend.image_data_format())
# bn_axis=1
# channel_ordering='channels_first'
# force channels-last ordering
tf.keras.backend.set_image_data_format('channels_last')
print(tf.keras.backend.image_data_format())
bn_axis=-1
channel_ordering='channels_last'

channels_last
channels_last


In [8]:
# data
# (args.config["features"]/2)
args.dataset='cifar10'
train_loader, test_loader = get_data_loaders(args, kwargs)

images_l = []
labels_l = []
for images, labels in test_loader:
    images = images.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    if 'channels_last' in channel_ordering:
        images = np.transpose(images, (0,2,3,1))
    images_l.extend(images)
    
    labels_l.extend(labels)

train_images_l = []
train_labels_l = []
for images, labels in train_loader:
    images = images.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    if 'channels_last' in channel_ordering:
        images = np.transpose(images, (0,2,3,1))
    train_images_l.extend(images)
    train_labels_l.extend(labels)

tf_test_dataset = tf.data.Dataset.from_tensor_slices( (images_l, labels_l) )
tf_test_dataset = tf_test_dataset.shuffle(buffer_size=100).batch(64)

tf_train_dataset = tf.data.Dataset.from_tensor_slices( (train_images_l, train_labels_l) )
tf_train_dataset = tf_train_dataset.batch(64)


Files already downloaded and verified
Files already downloaded and verified


In [13]:
# model
args.config = DATASET_CONFIGS[args.dataset]
print(args.config)
init_block_kernel=(3,3)

tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
# init_block
  tf.keras.layers.Input(shape=[args.config["seq"],args.config["features"],args.config["in_channels"]]),
#   tf.keras.layers.Input(shape=[args.config["in_channels"],args.config["features"],args.config["seq"]]),
  tf.keras.layers.Conv2D(filters=init_block_channel,kernel_size=init_block_kernel,
                        strides=init_block_stride,padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
# 1st layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=
  1,kernel_size=(3,3),
                        strides=strides[0],padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[0][0],kernel_size=(1,1),
                        padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
# 2nd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[1],padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[1][0],kernel_size=(1,1),
                        padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
# 3rd layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[2],padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[2][0],kernel_size=(1,1),
                        padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
# 4th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[3],padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[3][0],kernel_size=(1,1),
                        padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
# 5th layer of separable depthwise conv2d
  tf.keras.layers.DepthwiseConv2D(depth_multiplier=1,kernel_size=(3,3),
                        strides=strides[4],padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
  tf.keras.layers.Conv2D(filters=channels[4][0],kernel_size=(1,1),
                        padding='same',use_bias=False),
  tf.keras.layers.BatchNormalization(axis=bn_axis),
  tf.keras.layers.Activation('relu'),
# Note: Add dropout layer after all blocks and before pooling
  tf.keras.layers.Dropout(rate=0.4),
  tf.keras.layers.AveragePooling2D(pool_size=get_pool_size(strides, args.config),data_format=channel_ordering),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(units=args.config["classes"])
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

{'in_channels': 3, 'features': 32, 'seq': 32, 'classes': 10}


In [14]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 176)       4752      
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 176)       704       
_________________________________________________________________
activation (Activation)      (None, 16, 16, 176)       0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 8, 8, 176)         1584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 176)         704       
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 176)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 192)         3

In [ ]:
model.fit(tf_train_dataset, epochs=1, validation_data=tf_test_dataset)

782/782 [==============================] - 713s 910ms/step - loss: 1.8171 - sparse_categorical_accuracy: 0.3245 - val_loss: 1.4774 - val_sparse_categorical_accuracy: 0.4875


In [15]:
def get_dataset():
    """Generator function to produce representative dataset for post-training quantization."""
    # Use a few samples from the training set.
    for _ in range(100):
        img = iter(train_loader).next()[0].numpy()
        img = np.transpose( img , (0,2,3,1) )
        img = [(img.astype(np.float32))]
        yield img

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = get_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()
open("../data/tflite_model/MicroNet-AD-I176_S22_cifar10_int8.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/zz/mkr3s_81101_rh7kxcgvf7l80000gn/T/tmpo0c583i2/assets


252704

In [12]:
! xxd -i ../data/tflite_model/MicroNet-AD-I168_cifar10_int8.tflite > ../data/tflite_model/MicroNet-AD-I168_cifar10_int8.cc
